In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
tf.__version__

'2.8.0'

In [8]:
tf.test.is_built_with_cuda()

True

In [11]:
#Load the dataset
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [12]:
X_train.shape

(50000, 32, 32, 3)

In [13]:
y_train.shape


(50000, 1)

In [14]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [15]:
classes[y_train[3][0]]

'deer'

In [16]:
##Preprocessing: Scale images
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [17]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [18]:
y_train[0:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [19]:
y_train_categorical

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
##Model building and training
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [21]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [22]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [23]:
strategy.num_replicas_in_sync


1

In [24]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [ ]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 [==============================] - 49s 234ms/step - loss: 1.9842 - accuracy: 0.2958
Epoch 2/50
200/200 [==============================] - 47s 234ms/step - loss: 1.8198 - accuracy: 0.3625
Epoch 3/50
200/200 [==============================] - 47s 235ms/step - loss: 1.7578 - accuracy: 0.3877
Epoch 4/50
200/200 [==============================] - 47s 235ms/step - loss: 1.7146 - accuracy: 0.4052
Epoch 5/50
200/200 [==============================] - 47s 234ms/step - loss: 1.6803 - accuracy: 0.4155
Epoch 6/50
200/200 [==============================] - 47s 233ms/step - loss: 1.6513 - accuracy: 0.4254
Epoch 7/50
200/200 [==============================] - 47s 234ms/step - loss: 1.6261 - accuracy: 0.4352
Epoch 8/50
200/200 [==============================] - 47s 234ms/step - loss: 1.6038 - accuracy: 0.4434
Epoch 9/50
200/200 [==============================] - 47s 234ms/step - loss: 1.5838 - accuracy: 0.4504
Epoch 10/50
200/200 [==============================] - 47s 236ms/step - l

In [25]:
%%timeit -n1 -r1 
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 [==============================] - 59s 289ms/step - loss: 1.9795 - accuracy: 0.3000
Epoch 2/50
200/200 [==============================] - 58s 289ms/step - loss: 1.8159 - accuracy: 0.3636
Epoch 3/50
200/200 [==============================] - 59s 292ms/step - loss: 1.7545 - accuracy: 0.3875
Epoch 4/50
200/200 [==============================] - 59s 293ms/step - loss: 1.7116 - accuracy: 0.4039
Epoch 5/50
200/200 [==============================] - 59s 293ms/step - loss: 1.6770 - accuracy: 0.4169
Epoch 6/50
200/200 [==============================] - 59s 293ms/step - loss: 1.6478 - accuracy: 0.4273
Epoch 7/50
200/200 [==============================] - 59s 294ms/step - loss: 1.6224 - accuracy: 0.4366
Epoch 8/50
200/200 [==============================] - 59s 296ms/step - loss: 1.5999 - accuracy: 0.4449
Epoch 9/50
200/200 [==============================] - 59s 296ms/step - loss: 1.5798 - accuracy: 0.4519
Epoch 10/50
200/200 [==============================] - 59s 296ms/step - l